In [30]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import json
import pdb
import argparse
from easydict import EasyDict as edict
import jsonlines
import yaml
import random

import numpy as np
import matplotlib.pyplot as plt
import PIL
%matplotlib inline
import operator
import nltk

from tools.refer.refer import REFER

In [66]:
input_file_path = '../results/refcoco+_bert_base_6layer_6conect-pretrained-/val_result.json'
compare_file_path = '../results/refcoco+_bert_base_6layer_6conect-scratch-/val_result.json'
dataroot = '../data/referExpression'

input_file = json.load(open(input_file_path, 'r'))
compare_file = json.load(open(compare_file_path, 'r'))

refer = REFER(dataroot, dataset='refcoco+',  splitBy='unc')
ref_ids = refer.getRefIds(split='val')

print(len(ref_ids))
entries = []

for ref_id in ref_ids:
    ref = refer.Refs[ref_id]
    image_id = ref['image_id']
    ref_id = ref['ref_id']
    refBox = refer.getRefBox(ref_id)
    for sent, sent_id in zip(ref['sentences'], ref['sent_ids']):
        caption = sent['raw']
        entries.append(
            {"caption": caption, 'sent_id':sent_id, 'image_id':image_id, \
            "refBox": refBox, 'ref_id': ref_id}
            )
        
print(input_file[0])
print(entries[0])


loading dataset refcoco+ into memory...
creating index...
index created.
DONE (t=7.10s)
3805
{'id': 580957, 'target': 20, 'IOU': 0.8782015442848206}
{'caption': 'White bowl with vertical stripes', 'sent_id': 210, 'image_id': 580957, 'refBox': [468.3, 0.91, 171.7, 116.12], 'ref_id': 75}


In [ ]:
assert len(entries) == len(input_file)
words_freq = {}
words_win = {}
words_loss = {}
for i in range(len(entries)):
    input_IOU = input_file[i]['IOU']
    compare_IOU = compare_file[i]['IOU']
    caption = entries[i]['caption'].lower()
    tokens = nltk.word_tokenize(caption)
    pos_tag = nltk.pos_tag(tokens)
    for w in tokens:
        words_freq[w] = words_freq.get(w, 0) + 1
        
    if input_IOU > 0.5 and compare_IOU < 0.5:
        for i, w in enumerate(tokens):
            if pos_tag[i][1] in ['NN', 'NNS']:
                words_win[w] = words_win.get(w, 0) + 1
    elif input_IOU < 0.5 and compare_IOU > 0.5:
        for i, w in enumerate(tokens):
            if pos_tag[i][1] in ['NN', 'NNS']:
                words_loss[w] = words_loss.get(w, 0) + 1

for w, count in words_win.items():
    words_win[w] = count / max(float(words_freq[w]),10) #- words_loss.get(w,0)

sorted_words = sorted(words_win.items(), key=operator.itemgetter(1))
for i in range(len(words_win)):
    print(sorted_words[-i-1][1], sorted_words[-i-1][0])
   
print("global count")

sorted_words_freq = sorted(words_freq.items(), key=operator.itemgetter(1))
for i in range(len(words_freq)):
    print(sorted_words_freq[-i-1][1], sorted_words_freq[-i-1][0])
